### this is my master notebook for processing EGH data
- last edited March 23, 2020

In [1]:
#run this cell when online
!pip install xlrd

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## install dependencies and set data paths

In [1]:
import pandas as pd
import numpy as np

import os
import re
from collections import Counter

In [3]:
#use this cell when working online
path = '/floyd/home/ed-triage'
data_path = '/floyd/home/data'

In [100]:
#use this cell when working from home
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'
data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/egh'

In [3]:
#loading in the entire spreadsheet as a dataframe
clin = pd.read_excel(data_path + '/EGH ED visits FY1819_with additions.xlsx')

In [4]:
clin.shape

(85154, 27)

In [5]:
clin.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading'],
      dtype='object')

In [6]:
clin.dropna(subset = ['SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory', 'Triage Date & Time', 'Reg Date & Time'], inplace = True)
clin.shape

(63465, 27)

## The next few cells are for getting a feel of the data in the different columns and how they might be useful

In [9]:
discharge_count = Counter(clin['DischargeDisposition']); discharge_count

Counter({17: 88232,
         62: 263,
         63: 1419,
         7: 9283,
         6: 532,
         30: 217,
         40: 242,
         16: 499,
         13: 1199,
         64: 354,
         90: 23,
         12: 65,
         8: 139,
         14: 19,
         61: 49,
         9: 1,
         72: 7})

In [10]:
discharge_count2 = Counter(clin['DischargeDispositionDesc']); discharge_count2

Counter({'Discharge to private home, condo, apt without support service/referral': 88232,
         'Left at his/her own risk post-initial treatment': 263,
         'Left After Triage': 1419,
         'Admit to reporting facility as inpatient to another unit from amb care': 9283,
         'Admit to reporting facility as inpatient to SCU or OR from amb care': 532,
         'Transfer to Residential care': 217,
         'Transfer to Group/supportive living': 242,
         'Discharge to private home, condo, apt with support service/referral': 499,
         'Intra-facility transfer to ED': 1199,
         'Left After Initial Assessment': 354,
         'Transfer to correctional facility': 23,
         'Intra-facility transfer to day surgery': 65,
         'Transfer to another acute care facility directly from amb care': 139,
         'Intra-facility transfer to clinic': 19,
         'Left at his/her own risk following registration': 49,
         'Transfer to another non-acute care facility dir

In [11]:
comp_count = Counter(clin['PatientService']); comp_count

Counter({nan: 92747,
         'Orthopaedics': 138,
         'MH Child & Adolescent': 289,
         'INTENSIVE CARE UNIT': 170,
         'General Medicine': 3566,
         'Respirology': 410,
         'Genitourinary': 505,
         'General Surgery': 1069,
         'Cardiology': 1408,
         'Mental Health': 418,
         'Paediatrics': 823,
         'Palliative': 74,
         'Oncology': 322,
         'Neurology': 269,
         'ALC General Medicine': 96,
         'ALC Neurology': 45,
         'ALC Respirology': 13,
         'Acute Care for Elders': 29,
         'ALC Cardiology': 40,
         'Neonatal Retro Transfers': 4,
         'ALC Orthopaedics': 46,
         'AD PALLIATIVE': 14,
         'ALC General Surgery': 12,
         'Obstetrics': 19,
         'ALC Oncology': 4,
         'AD Respirology': 1,
         'AD Oncology': 5,
         'ALC ACUTE CARE FOR EDLERS': 2,
         'AD Cardiology': 1,
         'ALC Palliative': 3,
         'AD General Surgery': 1})

## refactoring some of the cells so they are clean and useful

### tabular data items

In [7]:
o2sat = [item if item[-2:] != 'ra' else item[:-2] for item in clin['O2Saturation_LastEDReading'].astype('str')]
o2sat = [np.nan if (len(item) <2 or len(item) >3) else float(item) for item in o2sat] 
o2sat = [np.nan if item == 'nan' else float(item) for item in o2sat] 
clin['o2sat'] = o2sat

In [8]:
pulse = clin['Pulse_LastEDReading']
pulse = [re.sub("[^0-9]", "", str(item)) for item in pulse]
pulse = [np.nan if item == '' else float(item) for item in pulse] 
#set(pulse)
clin['pulse'] = pulse

In [9]:
temp = clin['Temperature_LastEDReading']
temp = [re.sub("[^0-9]", "", str(item)) for item in temp]
temp = [np.nan if item == '' else float(item) for item in temp] 
temp = [item/10 if item > 100 else item for item in temp]
temp = [np.nan if item > 40  else item for item in temp]
#set(temp)
clin['temp'] = temp

In [11]:
def bptrans(bp):
    if pd.isnull(bp) or len(bp.split('/')) !=2:
        return [np.nan, np.nan]
    res = []
    for x in bp.split('/'):
        try:
            float(x)
            res =  [float(x) for x in bp.split('/')]
        except: 
            res =  [np.nan, np.nan]
        return res

In [12]:
clin['BP'] = clin.BloodPressure_LastEDReading.map(bptrans)

clin[['systolic', 'diastolic']] = pd.DataFrame(clin.BP.tolist(), index= clin.index)
clin["Gender"] = clin.GenderDesc.map(lambda x: 1 if x == "Male" else 0)

### clean subjective notes

In [13]:
#this is to clean the text in the subjective notes column (a little bit)
def fixencode(s):
    if pd.isnull(s):
        s = "None"
    s = s[14:]
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    s = s.replace('hrs', 'hours')
    s = s.replace('hx', 'history')
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('yrs', 'years')
    s = s.replace('lmp', 'last menstrual period')
    s = s.replace('w/', 'with')
    s = s.replace('@', 'at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = s.replace(' htn ', ' hypertension ')
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    #note that what's missing here is removal of punctuation
    return s

In [14]:
clin['CleanSubjectiveNotes'] = clin.SubjectiveNotes.map(fixencode)

### cleaning text of medical hx (for NLP)

In [15]:
#this is to clean the text in the medical history column (a little bit)
def fixmedhx(s):
    s = s[16:]
    s =s.replace('<LT><LF>',' ')
    s = s.replace('<LT>LF>',' ')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    s = s.replace('hrs', 'hours')
    s = s.replace('hx', 'history')
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('yrs', 'years')
    s = s.replace('lmp', 'last menstrual period')
    s = s.replace('w/', 'with')
    s = s.replace('@', 'at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics ')
    s = s.replace(' htn ', ' hypertension ')
    s = re.sub(r'\btyl\b', 'tylenol ', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant ',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant ',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub(r'#\s', 'fracture ', s)
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    #note that what's missing here is removal of punctuation
    return s

In [16]:
clin['CleanMedicalHistory'] = clin.MedicalHistory.map(fixmedhx)

### clean and split medical history into list of comorbidites

In [17]:
history = list(clin['MedicalHistory'].str[16:].astype('str'))

In [18]:
#this is to clean the text in the medical history column (a little bit)
def fixhx(s):
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    s = s.replace('hrs', 'hours')
    s = s.replace('hx', 'history')
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('yrs', 'years')
    s = s.replace('lmp', 'last menstrual period')
    s = s.replace('w/', 'with')
    s = s.replace('@', 'at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = s.replace(' htn ', ' hypertension ')
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub(r'#\s', 'fracture ', s)
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    #note that what's missing here is removal of punctuation
    return s

In [19]:
replacements = (',','<LT>LF>', '.')
drop_list = ('"', '?')
split_hist = []
for item in history:
    for r in replacements:
        item = item.replace(r, '::')
    
    item = item.split('::')
    
    item = [re.sub(r'\(.*?\)\ *', '', s) for s in item]
    
    item = [fixhx(s) for s in item]
    for d in drop_list:
        item = [x.replace(d,'') for x in item]
    
    item = [x.strip().lower() for x in item if x != 'nan']
    item = [x for x in item if len(x) > 2]
    
    item = list(filter(None, item))
    
    split_hist.append(item)
    

In [20]:
#rejoining them as a string with diagnoses separated by commas
join_hist = [','.join(item) for item in split_hist]

In [21]:
comorbids = [len(item) if item != ['no significant medical history'] else 0 for item in split_hist]

In [22]:
#replacing empty string with NaN
join_hist = [np.nan if item == 'nan' else item for item in join_hist]

In [23]:
len(join_hist)

63465

In [24]:
clin['pmhx'] = join_hist
clin['num_comorbids'] = comorbids

### exploring different diagnoses
- in here I initially made some code to make a data frame up of the top 6 diagnoses for each patient

In [81]:
diagnoses = Counter()
for hx in split_hist:
    for item in hx:
        diagnoses[item] += 1

In [82]:
#there are 20464 different medical histories used in the dataset!!!
len(diagnoses)

16707

In [83]:
#sorted diagnoses
diagnoses.most_common(40)

[('no significant medical history', 44103),
 ('hypertension', 14912),
 ('high cholesterol', 13371),
 ('niddm', 7171),
 ('high bp', 4756),
 ('depression', 4566),
 ('asthma', 4316),
 ('anxiety', 3493),
 ('thyroid', 3268),
 ('diabetes', 2866),
 ('hypothyroid', 2480),
 ('iddm', 2450),
 ('gerd', 2210),
 ('htn', 2120),
 ('acid reflux', 1719),
 ('arthritis', 1460),
 ('anemia', 1087),
 ('copd', 1035),
 ('schizophrenia', 838),
 ('kidney stones', 784),
 ('chf', 738),
 ('seizure', 698),
 ('atrial fibrillation', 688),
 ('immunizations up to date', 686),
 ('enlarged prostate', 679),
 ('gout', 655),
 ('cva', 635),
 ('adhd', 612),
 ('migraines', 586),
 ('dementia', 542),
 ('substance misuse', 537),
 ('high chol', 500),
 ('bipolar', 478),
 ('osteoporosis', 466),
 ('chronic back pain', 463),
 ('smoker', 431),
 ('fibromyalgia', 425),
 ('cholesterol', 411),
 ('pacemaker/cardioverter/defib', 394),
 ('pneumonia', 389)]

In [84]:
#this is a dictionary mapping the rank of the diagnosis to its name
dx_rank_dict = {item[0]:i+1 for i,item in enumerate(diagnoses.most_common())}

#this is a dictionary mapping the frequency of occurence to its name
dx_freq_dict = {item[0]:item[1]/len(diagnoses) for item in diagnoses.most_common()}

In [85]:
#trying to get a feel for how many diagoses are provided for each patient
split_hist_len = [len(item) for item in split_hist]
hxdf = pd.DataFrame(split_hist_len)

In [86]:
hxdf.describe()

,0
count,102543.000000
mean,1.759847
std,1.374310
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,27.000000


In [87]:
lengths = Counter(split_hist_len); lengths

Counter({1: 66562,
         5: 2807,
         6: 1307,
         2: 15334,
         3: 9536,
         8: 323,
         4: 5503,
         9: 167,
         7: 610,
         13: 20,
         0: 187,
         11: 42,
         10: 92,
         12: 27,
         17: 4,
         15: 11,
         14: 5,
         16: 5,
         27: 1})

In [88]:
#most patients have only a single diagnosis, 80% have 2 or less, 96% have 6 or less
for k,v in sorted(lengths.items()):
    print (k, '{0:2f}%'.format(v*100/len(split_hist_len)))

0 0.182363%
1 64.911306%
2 14.953727%
3 9.299513%
4 5.366529%
5 2.737388%
6 1.274587%
7 0.594872%
8 0.314990%
9 0.162859%
10 0.089718%
11 0.040958%
12 0.026330%
13 0.019504%
14 0.004876%
15 0.010727%
16 0.004876%
17 0.003901%
27 0.000975%


In [89]:
#this will create a list for each item where the top 6 diagnoses are listed in order
ordered_hist = []
for item in split_hist:
    #print (item)
    #print (len(item))
    top_6 = []
    for hx in item:
        #print (hx)
        #print (dx_rank_dict[hx])
        #print (dx_rank_dict[hx], hx)
        if item == 'nan':
            top_6.append(np.nan)
        else:
            top_6.append((dx_rank_dict[hx], hx))
    #print (top_6)
    sort = sorted(top_6)
    #print (sort)
    sort2 = [ii for i,ii in sort]
    #print(sort2)
    if len(sort2) > 6:
        sort2 = sort2[:5]
    if len(sort2) < 6:
        pad_req = 6-len(sort2)
        i = 0
        while i < pad_req:
            sort2.append(np.nan) #('n/a')
            i = i+1
        
    #print (top_6)
    #print (sorted(top_6))
    ordered_hist.append(sort2)
    #print ('next item')
    #print ()

In [90]:
medhx_df = pd.DataFrame(ordered_hist, columns = ['medhx' + str(i+1) for i in range(6)])
medhx_df.shape

(102543, 6)

In [91]:
ordered_hist[2000:2020]

[['no significant medical history', nan, nan, nan, nan, nan],
 ['high cholesterol', 'kidney stones', nan, nan, nan, nan],
 ['back pain', nan, nan, nan, nan, nan],
 ['cholesterol', 'benign polyps', nan, nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['high bp', 'hypothyroid', 'chf', nan, nan, nan],
 ['hypertension', 'prostate cancer', 'constipation', nan, nan, nan],
 ['depression', 'asthma', nan, nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['anxiety', 'pneumonia', 'bipolar disorder', nan, nan, nan],
 ['autistic', 'virus in the mouth', 'healed', nan, nan, nan],
 ['no significant medical history',
  'unable to get medical history',
  nan,
  nan,
  nan,
  nan],
 ['hypertension', 'high cholesterol', 'diabetes', nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['hypertension',
  'high cholesterol',
  'acid reflux',
  'pacemaker/cardiov

In [92]:
comorbids = []

for item in ordered_hist:
    res = [x for x in item if str(x) != 'nan']
    if len(res) == 0: comorbids.append(np.nan)
    else:
        res = [x for x in res if x != 'no significant medical history']
        #print (res, len(res))
        comorbids.append(len(res))

In [93]:
#so now I want to add this to the tabular data frame
tab_df['num_comorbids'] = comorbids

/Users/jjaskolkambp/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [94]:
clin['num_comorbids'] = comorbids

### some sanity checking

In [18]:
for item in clin['CleanSubjectiveNotes'].iloc[10000:10010]:
    print (item)

patient states feeling depressed for few months,last tuesday patient tried to harm herself by taking 10 extra strength tylenol. denies any intentions to harm herself now or any intention to harm others,
patient was playing around at school, was pushed by fellow student, patient felt a twisting sensation on her left knee and fell to floor. hurts to ambulate on, good csm
as per mother, patient was in the school bus yesterday and it was hit form behind. since then patient has been exp nausea, headaches and stomach aches. no vomits or diarrhea. no urinary or bowel symptoms. no neuro deficits noted. no resp distres.
sst fell on ladder this am,landed to right foot, complains of pain with swelling, sts no head injury, concerned abt his foot, sts took tylenol #3 2 tabs, abt 10 am.sts unable to wieght bear.
as per patient with pain to lateral left back (describes as stabbing pain) radiating across mid back x1 month. seen at wic and bw done and patient sent to emergency department with elevated 

## split the screening questions
- result is a df called: screening_df

In [25]:
qlist = ['Are you feeling feverish or have had shakes or chills in the last 24 hours?',
         'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
        'Do you have a new Rash?',
        'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
         'Have you travelled outside of Canada/USA in the last 3 weeks?',
         'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
         'Have you received Health Care in another country in the last 2 years?',
        'Do you have a new/worse cough or shortness of breath?',
         'If so, select all countries that apply',
        'If so, select all infectious diseases that apply']

In [26]:
#making a dict so that the indect of an item on the list, matches the question later
q_dict = {i:q for i,q in enumerate(qlist)}

In [27]:
q_dict

{0: 'Are you feeling feverish or have had shakes or chills in the last 24 hours?',
 1: 'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
 2: 'Do you have a new Rash?',
 3: 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
 4: 'Have you travelled outside of Canada/USA in the last 3 weeks?',
 5: 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
 6: 'Have you received Health Care in another country in the last 2 years?',
 7: 'Do you have a new/worse cough or shortness of breath?',
 8: 'If so, select all countries that apply',
 9: 'If so, select all infectious diseases that apply'}

In [28]:
#this is going to be a list made of up all the entries in the infection control screening column
#casting the results as string to manipulate later
qans = list(clin['InfectionControlScreening'].str[14:].astype('str'))

In [29]:
#this is going to give me a way to split the questions by finding where they occur in the field
split_points = []
for item in qans:
    temp_points = []
    for word in qlist:
        temp_points.append(item.find(word))
    split_points.append(temp_points)
    

In [30]:
#this is going to give me list of places to split each entry (using only the questions that are actually present)
real_points = []
for item in split_points:
    real_points.append([idx for idx in sorted(item) if idx > 0])

In [31]:
split_points[10000:10010]

[[241, 432, 400, 325, 0, 70, 558, 179, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1]]

In [32]:
#anything from the list above which would have been empty in the original data
#gives a list of -1's in split_points and nothing in real_points
real_points[10000:10010]

[[70, 179, 241, 325, 400, 432, 558],
 [70, 179, 241, 325, 400, 432, 558],
 [70, 179, 241, 325, 400, 432, 558],
 [70, 179, 241, 325, 400, 432, 558],
 [70, 179, 241, 325, 400, 432, 558],
 [70, 179, 241, 325, 400, 432, 558],
 [70, 179, 241, 325, 400, 432, 558],
 [70, 179, 241, 325, 400, 432, 558],
 [70, 179, 241, 325, 400, 432, 558],
 [70, 179, 241, 325, 400, 432, 558]]

In [33]:
print(qans[1000])
print()
print (split_points[1000])
print ()
print(real_points[1000])

Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>Do you have a new/worse cough or shortness of breath? Y<LT>LF>Are you feeling feverish or have had shakes or chills in the last 24 hours? N<LT>LF>Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>Do you have a new Rash? N

[241, -1, 400, 325, 0, 70, -1, 179, -1, -1]

[70, 179, 241, 325, 400]


In [34]:
#this should mutate qans so each item is now a list of q and a's
for idx in range(len(qans)):
    qans[idx] = [qans[idx][i : j] for i, j in zip([0] + real_points[idx], real_points[idx] + [None])]
    

In [35]:
qans[1000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Do you have a new/worse cough or shortness of breath? Y<LT>LF>',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N<LT>LF>',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>',
 'Do you have a new Rash? N']

In [36]:
qans[10000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Do you have a new/worse cough or shortness of breath? N<LT>LF>',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N<LT>LF>',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>',
 'Do you have a new Rash? N<LT>LF>',
 'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting? N<LT>LF>',
 'Have you received Health Care in another country in the last 2 years? N']

In [37]:
#this will mutate each item of each list in qans and drop the <LT>LF> if present
for q in qans:
    for idx in range(len(q)):
        if q[idx].find('<LT>LF>') != -1:
            q[idx] = q[idx][:-7]
        

In [38]:
qans[1000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? Y',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? N']

In [39]:
qans[10000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? N',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? N',
 'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting? N',
 'Have you received Health Care in another country in the last 2 years? N']

In [40]:
seg_list = []
for q in qans:
    #print ('initial:', q)
    test_question = q
    res = []
    for item in test_question:
        #print (item)
        for qq in qlist:
            if item.find(qq) != -1:
                #print ('question:',item[:len(qq)])
                #print ('answer:',item[len(qq)+1:])
                res.append([item[:len(qq)],item[len(qq)+1:] ])
    seg_list.append(res)
    #print ('\nresult:',res, '\n\n')

In [41]:
len(seg_list)

63465

In [42]:
seg_list2 = []
for q in qans:
    #print ('initial:', q)
    test_question = q
    res = {}
    for item in test_question:
        #print (item)
        for qq in qlist:
            if item.find(qq) != -1:
                question = item[:len(qq)]
                answer = item[len(qq)+1:]
                res[question] = answer
    seg_list2.append(res)

In [43]:
final_list = []
for q in range(len(split_points)):
    temp = []
    for i in range(len(split_points[q])):
        if split_points[q][i] == -1:
            #print('nan')
            temp.append('nan')
        else:
            #print (brief_list2[q][q_dict[i]])  # q_dict[i], i, brief_splits[q][i],
            temp.append(seg_list2[q][q_dict[i]])
    final_list.append(temp)
    #print ('\n next item \n')
            

In [44]:
len(final_list)

63465

In [45]:
final_list[100:110]

[['Y', 'N', 'N', 'Y', 'Y', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['nan', 'N', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['nan', 'N', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan']]

In [46]:
final_list[1000:1020]

[['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'nan', 'N', 'Y', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['Y', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'IND', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['Y', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan',

In [47]:
screening_df = pd.DataFrame(final_list, columns = qlist)
screening_df.head()

,Are you feeling feverish or have had shakes or chills in the last 24 hours?,Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?,Do you have a new Rash?,Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?,Have you travelled outside of Canada/USA in the last 3 weeks?,Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?,Have you received Health Care in another country in the last 2 years?,Do you have a new/worse cough or shortness of breath?,"If so, select all countries that apply","If so, select all infectious diseases that apply"
0,N,N,N,N,N,N,N,N,nan,nan
1,N,nan,N,nan,N,N,nan,N,nan,nan
2,N,nan,N,N,Y,N,nan,N,nan,nan
3,N,nan,N,N,N,N,nan,N,nan,nan
4,N,nan,N,N,Y,N,nan,N,IND,nan


In [48]:
screening_df.shape

(63465, 10)

In [50]:
screening_df.iloc[60000:60010]

,Are you feeling feverish or have had shakes or chills in the last 24 hours?,Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?,Do you have a new Rash?,Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?,Have you travelled outside of Canada/USA in the last 3 weeks?,Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?,Have you received Health Care in another country in the last 2 years?,Do you have a new/worse cough or shortness of breath?,"If so, select all countries that apply","If so, select all infectious diseases that apply"
60000,N,nan,nan,N,N,nan,nan,N,nan,nan
60001,Y,nan,nan,nan,N,nan,nan,Y,nan,nan
60002,N,nan,nan,Y,N,nan,nan,N,nan,nan
60003,N,nan,nan,nan,N,nan,nan,N,nan,nan
60004,N,nan,nan,N,N,nan,nan,N,nan,nan
60005,N,nan,nan,N,N,nan,nan,N,nan,nan
60006,N,nan,nan,Y,N,nan,nan,N,nan,nan
60007,N,nan,nan,Y,N,nan,nan,N,nan,nan
60008,N,nan,nan,N,N,nan,nan,N,nan,nan
60009,N,nan,nan,N,N,nan,nan,N,nan,nan


In [51]:
screening_df.replace('nan',np.NaN, inplace = True)

In [52]:
screening_df.shape

(63465, 10)

In [53]:
clin.shape

(63465, 38)

## making the new target cells

In [60]:
admit_count= Counter(clin['AdmitLocation']); admit_count

Counter({nan: 56881,
         'E.5TH': 614,
         'E.7TH East': 842,
         'E.7TH West': 385,
         'E.10.RESP': 590,
         'E.MAU': 1043,
         'E.OVE': 646,
         'E.8SURGERY': 696,
         'E.9th MED': 230,
         'E.6TH': 554,
         'E.CCU': 202,
         'E.ICU': 96,
         'E.OVEMH': 156,
         'E.9MEDICAL': 143,
         'E.4TH': 10,
         'E.8MEDFLEX': 67,
         'E.SCN': 2,
         'E.9TH EAST': 127,
         'E.9TH WEST': 171,
         'E.7MEDFLEX': 10})

In [61]:
clin['outcome'] = clin['AdmitLocation'].astype('str').map(
{'nan': 'discharge',
         'E.OVE': 'madmit',
         'E.9MEDICAL': 'madmit',
         'E.CCU': 'madmit',
         'E.10.RESP': 'madmit',
         'E.MAU': 'madmit',
         'E.ICU': 'ICU',
         'E.8SURGERY': 'sadmit',
         'E.6TH': 'madmit',
         'E.5TH': 'madmit',
         'E.OVEMH': 'madmit',
         'E.7TH West': 'madmit',
         'E.9th MED': 'madmit',
         'E.7TH East': 'madmit',
         'E.4TH': 'madmit',
         'E.8MEDFLEX': 'madmit',
         'E.SCN': 'madmit',
         'E.9TH EAST': 'madmit',
         'E.9TH WEST': 'madmit',
         'E.7MEDFLEX': 'madmit'})

In [62]:
#first target columns is based on the admitting location
clin['target'] = clin['outcome'].map({'discharge': 1, 'madmit': 2, 'sadmit': 3, 'ICU': 4})

In [63]:
outcome_count = Counter(clin['outcome']); outcome_count

Counter({'discharge': 56881, 'madmit': 5792, 'sadmit': 696, 'ICU': 96})

In [64]:
target_count = Counter(clin['target']); target_count

Counter({1: 56881, 2: 5792, 3: 696, 4: 96})

In [65]:
clin['outcome'] = clin['AdmitLocation'].astype('str').map(
{'nan': 'discharge',
         'E.OVE': 'madmit',
         'E.9MEDICAL': 'madmit',
         'E.CCU': 'madmit',
         'E.10.RESP': 'madmit',
         'E.MAU': 'madmit',
         'E.ICU': 'ICU',
         'E.8SURGERY': 'sadmit',
         'E.6TH': 'madmit',
         'E.5TH': 'madmit',
         'E.OVEMH': 'madmit',
         'E.7TH West': 'madmit',
         'E.9th MED': 'madmit',
         'E.7TH East': 'madmit',
         'E.4TH': 'madmit',
         'E.8MEDFLEX': 'madmit',
         'E.SCN': 'madmit',
         'E.9TH EAST': 'madmit',
         'E.9TH WEST': 'madmit',
         'E.7MEDFLEX': 'madmit'})

In [66]:
service_count = Counter(clin['service'])
service_count

Counter({'discharge': 56881, 'madmit': 5708, 'sadmit': 776, 'ICU': 100})

In [67]:
#second target is based on the admitting service
clin['target2'] = clin['service'].map({'discharge': 1, 'madmit': 2, 'sadmit': 3, 'ICU': 4})

In [68]:
target2_count = Counter(clin['target2']); target2_count

Counter({1: 56881, 2: 5708, 3: 776, 4: 100})

In [69]:
#creating this for later use to map the numbers in the dataframe column to their original meaning
out_dict = {'discharge': 1, 'madmit': 2, 'sadmit': 3, 'ICU': 4}
out_dict = {v:k for k,v in out_dict.items()}
out_dict

{1: 'discharge', 2: 'madmit', 3: 'sadmit', 4: 'ICU'}

In [70]:
#discharge vs. not target
clin['discharge'] = clin['outcome'].map({'discharge': 'discharge', 'madmit': 'admit', 'sadmit': 'admit', 'ICU': 'admit'})

In [71]:
clin['target3'] = clin['outcome'].map({'discharge': 1, 'madmit': 0, 'sadmit': 0, 'ICU': 0})

In [72]:
Counter(clin['discharge']), Counter(clin['target3'])

(Counter({'discharge': 56881, 'admit': 6584}), Counter({1: 56881, 0: 6584}))

In [73]:
#a target for trying to tell apart the inpatient disposition, only in admitted patients
clin['dispo']= clin['outcome'].map({'discharge': np.nan, 'madmit': 'madmit', 'sadmit': 'sadmit', 'ICU': 'ICU'})

In [74]:
clin['target4'] = clin['dispo'].map({np.nan:np.nan, 'madmit': 1, 'sadmit': 2, 'ICU': 3})
#for some reason a can't make a counter out of this without it crashing so I've stopped trying

In [75]:
Counter(clin['dispo']), Counter(clin['target4'].dropna())

(Counter({nan: 56881, 'madmit': 5792, 'sadmit': 696, 'ICU': 96}),
 Counter({1.0: 5792, 2.0: 696, 3.0: 96}))

In [76]:
#gonna make a target that is ICU vs other
clin['ICUvsother'] = clin['PatientService'][clin['PatientService'] == 'INTENSIVE CARE UNIT']
#clin['PatientService']

In [77]:
clin['target5'] = clin['ICUvsother'].map({'INTENSIVE CARE UNIT':1, np.nan:0})

### merging to a full dataset

In [78]:
clin.shape

(63465, 48)

In [79]:
screening_df.shape

(63465, 10)

In [80]:
full = pd.concat([clin,screening_df], axis = 1).reindex(clin.index); full.shape

(63465, 58)

In [81]:
full.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading', 'o2sat', 'pulse',
       'temp', 'BP', 'systolic', 'diastolic', 'Gender', 'CleanSubjectiveNotes',
       'CleanMedicalHistory', 'pmhx', 'num_comorbids', 'outcome', 'target',
       'service', 'target2', 'discharge', 'target3', 'dispo', 'target4',
       'ICUvsother', 'target5',
       'Are you feeling feverish or have had shakes or chills in the last 24 hours?',
      

## splitting into a bunch of sub dataframes
- later will save them as their own csv files
- loading the excel file is very slow, but loading csv files is faster so I'm making a bunch of dataframes of subsets of data for easy saving/loading/combining

In [67]:
tab_df = clin[['ID', 'TriageLevel',
       'AgeInYrs', 'GenderDesc', 'DischargeDisposition',
       'PresentingComplaint', 'PresentingComplaintDesc', 'AdmitLocation', 'PatientService',
       'BloodPressure_LastEDReading','systolic', 'diastolic','temp','pulse','o2sat']]

In [68]:
target_df = clin[['outcome','target', 'service','target2', 'discharge', 'target3','dispo','target4', 'ICUvsother', 'target5']]

In [69]:
date_df = clin[['Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'Left ED Date & Time']]

## now I'm going to make a couple of balanced datasets

In [82]:
Counter(clin.target), Counter(clin.target3)

(Counter({1: 56881, 2: 5792, 3: 696, 4: 96}), Counter({1: 56881, 0: 6584}))

In [83]:
admit_df = clin[clin.target3 == 0]

dc_df = clin[clin.target3==1]
dc_df = dc_df.sample(n= len(admit_df))  #this is the number of admissions from target 3



In [84]:
balanced_df1 = pd.concat([admit_df, dc_df], axis = 0)

In [85]:
icu_df = clin[clin.target2 == 4]
madmit_df = clin[clin.target2 ==3].sample(n = len(icu_df))
sadmit_df = clin[clin.target2 ==2].sample(n = len(icu_df))
disc_df = clin[clin.target2 == 1].sample(n = len(icu_df))

In [86]:
#this is a dataframe with equal numbers of all 4 classes
balanced_df2 = pd.concat([icu_df, madmit_df, sadmit_df, disc_df], axis = 0)

In [88]:
non_icu_df = clin[clin.target2 !=4].sample(n = len(icu_df))

In [89]:
balanced_df3 = pd.concat([icu_df, non_icu_df], axis = 0)

In [90]:
balanced_df3.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading', 'o2sat', 'pulse',
       'temp', 'BP', 'systolic', 'diastolic', 'Gender', 'CleanSubjectiveNotes',
       'CleanMedicalHistory', 'pmhx', 'num_comorbids', 'outcome', 'target',
       'service', 'target2', 'discharge', 'target3', 'dispo', 'target4',
       'ICUvsother', 'target5'],
      dtype='object')

## saving dataframes to csv files

In [101]:
balanced_df1.to_csv(data_path + '/balanced_admit_dc_nlp_data.csv')

balanced_df2.to_csv(data_path +'/balanced_4cls_nlp_data.csv')

balanced_df3.to_csv(data_path +'/balanced_icuvsother_nlp_data.csv')

full.to_csv(data_path +'/complete_clean_egh_data.csv')


### now gonna try to mix datasets

In [121]:
#first reload the bch dataset
bch_data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/bch'
clin2 = pd.read_csv(bch_data_path +'/complete_clean_bch_data.csv', index_col = 0)
clin2.shape

/Users/jjaskolkambp/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (49,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(102492, 58)

In [122]:
#now reorder the EGH dataset columns to match the BCH dataset columns
full = full.reindex(columns=clin2.columns)

In [123]:
#now add a column to each indicating the site
full['site'] = 'EGH'
clin2['site'] = 'BCH'

In [125]:
#now merge them
combo_df = pd.concat([full, clin2], axis = 0)
combo_df.shape

(165957, 59)

### gonna make balanced combo datasets

In [128]:
Counter(combo_df.target), Counter(combo_df.target3)

(Counter({1.0: 149578, 2.0: 13839, 3.0: 2186, 4.0: 354}),
 Counter({1.0: 149578, 0.0: 16379}))

In [129]:
admit_df = combo_df[combo_df.target3 == 0]

dc_df = combo_df[combo_df.target3==1]
dc_df = dc_df.sample(n= len(admit_df))  #this is the number of admissions from target 3



In [130]:
combo_balanced_df1 = pd.concat([admit_df, dc_df], axis = 0)

In [131]:
icu_df = combo_df[combo_df.target2 == 4]
madmit_df = combo_df[combo_df.target2 ==3].sample(n = len(icu_df))
sadmit_df = combo_df[combo_df.target2 ==2].sample(n = len(icu_df))
disc_df = combo_df[combo_df.target2 == 1].sample(n = len(icu_df))

In [132]:
#this is a dataframe with equal numbers of all 4 classes
combo_balanced_df2 = pd.concat([icu_df, madmit_df, sadmit_df, disc_df], axis = 0)

In [133]:
non_icu_df = combo_df[combo_df.target2 !=4].sample(n = len(icu_df))

In [134]:
combo_balanced_df3 = pd.concat([icu_df, non_icu_df], axis = 0)

In [136]:
combo_balanced_df3.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading', 'o2sat', 'pulse',
       'temp', 'CleanSubjectiveNotes', 'BP', 'systolic', 'diastolic', 'Gender',
       'outcome', 'target', 'service', 'pmhx', 'num_comorbids', 'target2',
       'discharge', 'target3', 'dispo', 'target4', 'ICUvsother', 'target5',
       'CleanMedicalHistory',
       'Are you feeling feverish or have had shakes or chills in the last 24 hours?',
      

In [139]:

combo_data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/combo'

combo_balanced_df1.to_csv(combo_data_path + '/balanced_admit_dc_combo_data.csv')

combo_balanced_df2.to_csv(combo_data_path +'/balanced_4cls_combo_data.csv')

combo_balanced_df3.to_csv(combo_data_path +'/balanced_icuvsother_combo_data.csv')

combo_df.to_csv(combo_data_path +'/complete_clean_combo_data.csv')



In [ ]:
#need to create a set of validation indices to use across all databunches...

#can steal this code from another notebook...

In [ ]:
# make the date columns
date_columns = ['Triage Date & Time', 'Reg Date & Time']
for col in date_columns:
    add_datepart(clin, col, drop = False, time = True)
    add_cyclic_datepart(clin, col, time = True)

In [19]:
#need to reindex

new_index = np.arange(len(clin))

clin.index = new_index